In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

params['dataset'] = 'sent140'
#params['dataset'] = 'nist'
#params['model'] = 'mlp'
params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['allow_empty'] = True
  params['agg_lr'] = 0
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = True
  params['fesem'] = False
    
  """
  We immplement IFCA and FeSEM base on grouprox,
  Some configurations should be flush for above two CFL frameworks
  """
  if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0.0 # IFCA and FeSEM don't use this parameter

"""
Common parameters
"""
params['num_rounds'] = 800
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
    
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = info2 +  '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-sent140-stacked_lstm-IFCA-NumG5-Agglr0.0-ProximalF-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/ho

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
772 Clients in Total
243861 Parameters in Total
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


In [4]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

'\nTest Ternary Similarity\n'

In [5]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

'\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(7,7))\nplt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)\nplt.show()\n'

In [ ]:
server.train()

Training with 20 workers ---
Cold start clients takes 0.0009734630584716797s seconds
Scheduling clients takes 3.526742696762085s seconds
Group 0 is empty.
Group 1, clients ['steph_davies', 'TinaS71', 'coriluvnthedon', 'Mrs_NickJ07', 'mneylon', 'davidj505', 'MacSheikh']
Group 2, clients ['m0po', 'AvonteNikole', 'WooopJess']
Group 3, clients ['janabelle_xo', 'marianaguidil']
Group 4, clients ['debbier93', 'AllTheSausages', 'maddisondesigns', 'Japh', 'weelissa', 'nyc_specialist', 'schaeferj89', 'weeps']


/home/lab/workspace/GrouProx/flearn/trainers/grouprox.py:438: RuntimeWarning: invalid value encountered in double_scalars
  test_acc = np.sum(stats[3])*1.0/np.sum(stats[2])
/home/lab/workspace/GrouProx/flearn/trainers/grouprox.py:446: RuntimeWarning: invalid value encountered in double_scalars
  train_acc = np.sum(stats_train[3])*1.0/np.sum(stats_train[2])
/home/lab/workspace/GrouProx/flearn/trainers/grouprox.py:448: RuntimeWarning: invalid value encountered in double_scalars
  train_loss = np.dot(stats_train[4], stats_train[2])*1.0/np.sum(stats_train[2])
Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Group 0
At round 0 accuracy: nan
At round 0 training accuracy: nan
At round 0 training loss: nan
Group 1
At round 0 accuracy: 0.8051948051948052
At round 0 training accuracy: 0.7918088737201365
At round 0 training loss: 0.6646200078746157
Group 2
At round 0 accuracy: 0.6296296296296297
At round 0 training accuracy: 0.7474747474747475
At round 0 training loss: 0.6846140737485404
Group 3
At round 0 accuracy: 0.45454545454545453
At round 0 training accuracy: 0.6547619047619048
At round 0 training loss: 0.6826283250536237
Group 4
At round 0 accuracy: 0.6410256410256411
At round 0 training accuracy: 0.6305084745762712
At round 0 training loss: 0.6850677053807146
At round 0 mean test accuracy: 0.6813725490196079 mean train accuracy: 0.7094682230869002 mean train loss: 0.676973032425659                     number of test client: 20
The client-group discrepancy are: [37.81632548  0.         37.81879144 37.7426589  37.80998022 37.84337905]
Begin group  1 training


Epoch:   5%|▌         | 1/20 [00:00<00:02,  8.46it/s] 

Begin group  2 training


...

Epoch:   5%|▌         | 1/20 [00:00<00:02,  8.77it/s] 

Begin group  3 training


Epoch:   5%|▌         | 1/20 [00:00<00:02,  8.50it/s] 

Begin group  4 training


Training groups takes 82.227543592453s seconds
Scheduling clients takes 3.0837554931640625s seconds
Group 0, clients ['aarthycrazy', 'SandiNJ', 'AussieGal999']
Group 1, clients ['perpetualspiral']
Group 2, clients ['webmaster_paul', 'sophieholly', 'wickedground', 'xoHerbieox', 'taluta', 'witnessamiracle', 'Nikkers', 'KJL912', 'dreadw', 'unitechy']
Group 3, clients ['tweetpet']
Group 4, clients ['AreonLee', 'RGM77', 'ImmaChocoholic', 'shawnlimtianjun', 'toosweet4rnr']


In [ ]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""